# Intall Libraries, Run utlity functions, and import the queries (raw topics)

In [ ]:
%pip install stanza

In [ ]:
from pandas.core.groupby.generic import reconstruct_func
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import stanza
# download and initialize the CRAFT pipeline
stanza.download('en', package='craft')
nlp = stanza.Pipeline('en', package='craft')

## Select the collection and its related topics to create the reformulated topics

In [ ]:
collection = 'trec_clc' #trec_clc for trec2021 clinical or cds_clinical for the remaining 3 collections as they share the same topics

if collection == 'trec_clc':
  path_to_topics = './experiments/topics/trec_clc/topics2021.txt'
  path_to_save_entities = './experiments/topics/trec_clc/extracted_med_entities/'

elif collection == 'cds_clinical':
  path_to_topics = './topics-2014_2015-description.topics'
  path_to_save_entities = './experiments/topics/cds_clinical/extracted_med_entities/'

else:
  raise Exception("Invalid selection of topics. Use appropriate collections (trec_clc for TREC 2021 clinical. cds_clinical for cds and clinical collections)")

## Loading the associated topics

In [ ]:
# getting trec 2021 Data
def get_trec(path):
  topics = pd.read_csv(path)
  return topics

# parsing clinical and cds data
def parse_xml(path):
  with open(path, 'r') as document:
    d = document.read()
    soup = BeautifulSoup(d, 'xml')
    qid = soup.find_all('NUM')
    query = soup.find_all('TITLE')
    lq = []
    for i in qid: 
      lq.append(i.text)
    ld = []
    for x in query: 
      ld.append(x.text)
  topics = pd.DataFrame({'qid': lq,'query': ld})
  return topics

if collection =='trec_clc':
  topics = get_trec('/content/drive/MyDrive/data_colab/table_topics2021.csv')
  topics.head()
elif collection == 'cds_clinical':
  topics = parse_xml('/content/drive/MyDrive/data_colab/topics-2014_2015-description.topics')
  topics.head()

# Stanza implementation

In [ ]:
# extract named entities from the query
def query_to_ne(query,ent_type):
  l = []
  proc = nlp(query)
  for ent in proc.entities:
    if ent.type == ent_type:
      l.append(ent.text)
  return l

In [ ]:
topics['DISEASE'] = topics['query'].apply(query_to_ne, ent_type="DISEASE")
topics['CHEMICAL'] = topics['query'].apply(query_to_ne, ent_type="CHEMICAL")

In [ ]:
topics.head()

In [ ]:
topics["CHEMICAL"] = topics["CHEMICAL"].apply(lambda y: np.nan if len(y)==0 else y)
topics["DISEASE"] = topics["DISEASE"].apply(lambda x: np.nan if len(x)==0 else x)

In [ ]:
path_to_save_entities = path_to_save_entities+'dis_chem_stanza'+'.csv'
topics.to_csv(path_to_save_entities,index=False)